# NLP PROJECT : SPAM DETECTION

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

# Spark Session

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Spams').getOrCreate()

23/08/21 18:28:56 WARN Utils: Your hostname, Blade-15-Base-Model resolves to a loopback address: 127.0.1.1; using 192.168.0.22 instead (on interface wlo1)
23/08/21 18:28:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/21 18:28:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/21 18:28:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Importing Data

In [5]:
import urllib.request
import zipfile
import os

# Define the URL of the ZIP file
zip_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"

# Download and extract the ZIP file
zip_file_path = "smsspamcollection.zip"
extracted_folder_path = "smsspamcollection"
urllib.request.urlretrieve(zip_url, zip_file_path)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# Read the CSV file into a DataFrame
csv_path = os.path.join(extracted_folder_path, "SMSSpamCollection")
df = spark.read.csv(csv_path, sep="\t", header=False, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [8]:
df.columns

['_c0', '_c1']

In [9]:
# Renaming columns
df = df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")

# Show the DataFrame with renamed columns
df.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



# Preprocessing

### 1 Create a new length feature

In [6]:
from pyspark.sql.functions import length

In [10]:
df = df.withColumn('length',length(df['text']))

In [11]:
df.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



## 2 Distribution

In [12]:
# Pretty Clear Difference
df.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [13]:
# Pretty Clear Difference
df.groupby('class').count().show()

+-----+-----+
|class|count|
+-----+-----+
|  ham| 4827|
| spam|  747|
+-----+-----+



## 3 Feature Transformation

In [14]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

#### TOKENIZATION

Takes the sentence, makes it into a list of tokens or words

In [15]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")

#### REMOVING STOP WORDS

Takes the list of tokens, removes the words that aren't going to be usefull for analysis

In [16]:
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')

#### COUNT VECTOZERITION

In [17]:
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')

#### IDF

In [18]:
idf = IDF(inputCol="c_vec", outputCol="tf_idf")

#### STRING INDEXER

In [19]:
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

#### VECTOR ASSEMBLER

In [20]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [21]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

# MODELING

We'll use Naive Bayes, but feel free to play around with this choice!

In [23]:
from pyspark.ml.classification import NaiveBayes

In [24]:
# Use defaults
nb = NaiveBayes()

# PIPELINE

In [25]:
from pyspark.ml import Pipeline

In [26]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [28]:
cleaner = data_prep_pipe.fit(df)

In [29]:
clean_data = cleaner.transform(df)

In [31]:
clean_data.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,301,...|(13423,[0,24,301,...|(13424,[0,24,301,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,8

In [32]:
clean_data = clean_data.select(['label','features'])
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



# SPLIT TRAIN TEST

In [33]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [34]:
spam_predictor = nb.fit(training)

23/08/21 18:56:10 WARN DAGScheduler: Broadcasting large task binary with size 1164.9 KiB
23/08/21 18:56:10 WARN DAGScheduler: Broadcasting large task binary with size 1144.3 KiB


In [35]:
test_results = spam_predictor.transform(testing)

In [36]:
test_results.show()

23/08/21 18:56:39 WARN DAGScheduler: Broadcasting large task binary with size 1370.3 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1062.4583305547...|[1.0,4.6459593227...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-570.85214799250...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-447.65688091402...|[1.0,2.3359693854...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-1006.0361498109...|[1.0,9.9846428847...|       0.0|
|  0.0|(13424,[0,1,30,11...|[-629.16233983035...|[1.0,7.9353609792...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1155.1777153230...|[1.24730547521330...|       1.0|
|  0.0|(13424,[0,1,150,1...|[-254.74959644796...|[0.21250879028488...|       1.0|
|  0.0|(13424,[0,1,500,5...|[-317.53555358539...|[0.99999999999986...|       0.0|
|  0.0|(13424,[0,2,4,5,7...|[-993.83147708575...|[1.0,4.5129497009...|       0.0|
|  0.0|(13424,[0

# Evaluation

In [37]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [38]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

23/08/21 18:57:07 WARN DAGScheduler: Broadcasting large task binary with size 1374.9 KiB


Accuracy of model at predicting spam was: 0.9286781218599591


# <span style="color:green">NOT BAD KID